# Import Libraries

In [18]:
# Import all libraries 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta
from dateutil import parser
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
import os
from sklearn import metrics
from tensorflow.keras.utils import plot_model
import pydot
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'pydot'

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.naive_bayes import MultinomialNB, CategoricalNB
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC, SVC

In [ ]:
master_df = pd.read_csv('processed_data.csv')
master_df.head()

# 1. Regression

## Linear Regression

In [ ]:
dummy_topics = pd.get_dummies(master_df['topic'])

X_lr = pd.concat([X.loc[:, ['compound', 'favorites', 'retweets']], dummy_topics], axis=1)
y_lr = master_df.loc[:, '60mins_price_diff_perc']*100

In [2]:
# Removing correlated features

correlated_features = set()
correlation_matrix = X_lr.corr()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            
X_lr.drop(labels=correlated_features, axis=1, inplace=True)

In [ ]:
# Train test split and resampling

X_train_lr, X_test_lr, y_train_lr, y_test_lr = model_selection.train_test_split(X_lr, y_lr, test_size = 0.33, random_state = 2020)

smote = SMOTE(random_state=0, sampling_strategy='not majority')
X_train_lr, y_train_lr = smote.fit_sample(X_train_lr, y_train_lr)

In [ ]:
# Normalize all values
X_train_lr = preprocessing.scale(X_train_lr)
y_train_lr = preprocessing.scale(y_train_lr)
X_test_lr = preprocessing.scale(X_test_lr)
y_test_lr = preprocessing.scale(y_test_lr)

# Linear regression
regr = linear_model.LinearRegression()
regr.fit(X_train_lr, y_train_lr)

y_pred_lr = regr.predict(X_test_lr)
print('Linear metrics for stock price 60 mins after tweet')
print('Linear score:', regr.score(X_test_lr, y_test_lr))
print('Linear MSE', metrics.mean_squared_error(y_test_lr, y_pred_lr))
print(regr.coef_, '\n')

print('-'*64)

num_C = 5
C = [1.0] * num_C
for i in range(num_C):
    C[i] = pow(10, i-5)
print('Ridge and Lasso metrics for stock price 60 mins after tweet')
for i in range(5):
    # Lasso Regression
    lasso = linear_model.Lasso(alpha = C[i])
    lasso.fit(X_train_lr, y_train_lr)
    y_pred = regr.predict(X_test_lr)
    print('Alpha = ', C[i])
    print('Lasso score:', lasso.score(X_test_lr, y_test_lr))
    print('Lasso MSE', metrics.mean_squared_error(y_test_lr, y_pred_lr))
    print('Lasso coefs:', lasso.coef_, '\n')
    
    # Ridge regression
    ridge = linear_model.Ridge(alpha = C[i])
    ridge.fit(X_train_lr, y_train_lr)
    y_pred_lr = regr.predict(X_test_lr)
    print('Ridge score:', ridge.score(X_test_lr, y_test_lr))
    print('Ridge MSE', metrics.mean_squared_error(y_test_lr, y_pred_lr))
    print('Ridge coefs:', ridge.coef_, '\n')
    print('-'*64)

## LSTM Regression

### Model A: Word Vectors as Input

Method 1: Word2Vec

In [ ]:
X_lstm_reg_a_w2v = master_df.loc[:, 'lstm_text']
y_lstm_reg_a_w2v = master_df.loc[:, '60mins_price_diff_perc']*100

In [ ]:
X_train_lstm_reg_a_w2v, X_test_lstm_reg_a_w2v, y_train_lstm_reg_a_w2v, y_test_lstm_reg_a_w2v = train_test_split(X_lstm_reg_a_w2v, y_lstm_reg_a_w2v, test_size=0.33, random_state=2020)

In [ ]:
# Add words into corpus list

lstm_reg_a_corpus_list = []

for i in X_train_lstm_reg_a_w2v:
    lstm_reg_a_corpus_list.append(i.split())

In [ ]:
# Train Word2Vec

lstm_reg_a_w2v_model = Word2Vec(lstm_reg_a_corpus_list, min_count=1, size=100)
lstm_reg_a_w2v_weights = lstm_reg_a_w2v_model.wv.vectors

In [ ]:
# Find length of longest sentence (for padding later on)

lstm_reg_a_w2v_num_words = [len(i) for i in lstm_reg_a_w2v_corpus_list]
lstm_reg_a_w2v_longest_sentence_len = max(lstm_reg_a_w2v_num_words)

In [ ]:
# Pad sentences that are shorter than length of longest sentence in training data

def word2vec_sentence_to_indices_padded(sentences, longest_sentence_len, word2vec_model):
    result = []
    for sentence in sentences:
        indices = []
        sentence_splitted = sentence.split()
        for word in sentence_splitted:
            if word in word2vec_model.wv.vocab:
                indices.append(word2vec_model.wv.vocab[word].index)
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='pre')

In [ ]:
X_train_lstm_reg_a_w2v_padded = word2vec_sentence_to_indices_padded(X_train_lstm_reg_a_w2v, lstm_reg_a_w2v_longest_sentence_len, lstm_reg_a_w2v_model)
X_test_lstm_reg_a_w2v_padded = word2vec_sentence_to_indices_padded(X_test_lstm_reg_a_w2v, lstm_reg_a_w2v_longest_sentence_len, lstm_reg_a_w2v_model)

In [3]:
def create_lstm_reg_a_w2v(pretrained_weights, longest_sentence_len):
    vocab_size, embedding_size = pretrained_weights.shape
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False))  
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.2,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.2,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='tanh'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [ ]:
lstm_reg_a_w2v_ann = create_lstm_reg_a_w2v(lstm_reg_a_w2v_weights, lstm_reg_a_w2v_longest_sentence_len)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
lstm_reg_a_w2v_ann.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
lstm_reg_a_w2v_ann.summary()

In [ ]:
plot_model(
    lstm_reg_a_w2v_ann,
    to_file="lstm_reg_a_w2v_ann.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB"
)

In [ ]:
now = dt.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

lstm_reg_a_w2v_checkpoint_filepath = f'./lstm_reg_a_w2v/lstm_reg_a_w2v_{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

lstm_reg_a_w2v_history = lstm_reg_a_w2v_ann.fit(X_train_lstm_reg_a_w2v_padded, y_train_lstm_reg_a_w2v, validation_split=0.33, epochs=50, callbacks=[model_checkpoint_callback])

In [ ]:
lstm_reg_a_w2v_ann_loaded = create_lstm_reg_a_w2v(lstm_reg_a_w2v_weights, lstm_reg_a_w2v_longest_sentence_len)
lstm_reg_a_w2v_ann_loaded.load_weights(lstm_reg_a_w2v_checkpoint_filepath)
lstm_reg_a_w2v_ann_loaded.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [ ]:
dev_loss, dev_acc = lstm_reg_a_w2v_ann_loaded.evaluate(X_test_lstm_reg_a_w2v_padded, y_test_lstm_reg_a_w2v, verbose=1)

print(f"Training MSE: {np.sqrt(metrics.mean_squared_error(y_train_lstm_reg_a_w2v, lstm_reg_a_w2v_ann_loaded.predict(X_train_lstm_reg_a_w2v_padded)))}")
print(f"Test MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_a_w2v, lstm_reg_a_w2v_ann_loaded.predict(X_test_lstm_reg_a_w2v_padded)))}")
print(f"Test R^2: {metrics.r2_score(y_test_lstm_reg_a_w2v, lstm_reg_a_w2v_ann_loaded.predict(X_test_lstm_reg_a_w2v_padded))}")
print(f"Baseline MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_a_w2v, 0*y_test_lstm_reg_a_w2v))}")

In [ ]:
# summarize history for loss
plt.plot(lstm_reg_a_w2v_history.history['loss'])
plt.plot(lstm_reg_a_w2v_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Method 2: Glove

In [ ]:
X_lstm_reg_a_glove = master_df.loc[:, 'lstm_text']
y_lstm_reg_a_glove = master_df.loc[:, '60mins_price_diff_perc']*100

In [ ]:
X_train_lstm_reg_a_glove, X_test_lstm_reg_a_glove, y_train_lstm_reg_a_glove, y_test_lstm_reg_a_glove = train_test_split(X_lstm_reg_a_glove, y_lstm_reg_a_glove, test_size=0.33, random_state=2020)

In [ ]:
embeddings_index = {}
f = open('glove/glove.twitter.27B.50d.txt', encoding='utf8')
glove_vocab = []
glove_vocab_index = {}
count = 0
for line in f:
    values = line.split()
    word = values[0]
    glove_vocab.append(word)
    glove_vocab_index[word] = count
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    count += 1
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(nb_words=None)
tokenizer.fit_on_texts(X_train_lstm_reg_a_glove)
sequences = tokenizer.texts_to_sequences(X_train_lstm_reg_a_glove)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
def glove_sentence_to_indices_padded(sentences, longest_sentence_len):
    global glove_vocab
    result = []
    for sentence in sentences:
        indices = []
        try:
            sentence_splitted = sentence.split()
        except:
            continue
            
        for word in sentence_splitted:
            if word in glove_vocab:
                indices.append(glove_vocab_index[word])
        result.append(indices)
    return keras.preprocessing.sequence.pad_sequences(result, maxlen=longest_sentence_len, padding='post')

In [ ]:
X_train_lstm_reg_a_glove_padded = glove_sentence_to_indices_padded(X_train_lstm_reg_a_glove, lstm_reg_a_w2v_longest_sentence_len)
X_test_lstm_reg_a_glove_padded = glove_sentence_to_indices_padded(X_test_lstm_reg_a_glove, lstm_reg_a_w2v_longest_sentence_len)

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
count = 0
skipped_words = []
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        count += 1
        skipped_words.append(word)
        
vocab_size_glove, embedding_size_glove = embedding_matrix.shape

embedding_layer_glove = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=lstm_reg_a_w2v_longest_sentence_len,
                            trainable=False)

In [ ]:
def create_lstm_reg_a_glove(longest_sentence_len):
    global embedding_layer_glove
    model = tf.keras.Sequential()
    model.add(layers.Input(shape=longest_sentence_len, dtype='int32'))
    model.add(embedding_layer_glove)
    model.add(layers.LSTM(4, return_sequences=True, name='LSTM1'))
    model.add(layers.Dropout(0.2,name='Dropout1'))
    model.add(layers.LSTM(4, return_sequences=False, name='LSTM2'))
    model.add(layers.Dropout(0.2,name='Dropout2'))
    model.add(layers.Dense(4,name='Dense',activation='tanh'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1,activation='linear'))
    return model

In [ ]:
lstm_reg_a_glove_ann = create_lstm_reg_a_glove(lstm_reg_a_w2v_longest_sentence_len)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
lstm_reg_a_glove_ann.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
lstm_reg_a_glove_ann.summary()

In [ ]:
plot_model(
    lstm_reg_a_glove_ann,
    to_file="lstm_reg_a_glove_ann.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB"
)

In [ ]:
now = dt.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

lstm_reg_a_glove_checkpoint_filepath = f'./lstm_reg_a_glove/lstm_reg_a_glove_{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

lstm_reg_a_glove_history = lstm_reg_a_glove_ann.fit(X_train_lstm_reg_a_glove_padded, y_train_lstm_reg_a_glove, validation_split=0.33, epochs=50, callbacks=[model_checkpoint_callback])

In [ ]:
lstm_reg_a_glove_ann_loaded = create_lstm_reg_a_glove(lstm_reg_a_w2v_longest_sentence_len)
lstm_reg_a_glove_ann_loaded.load_weights(lstm_reg_a_glove_checkpoint_filepath)
lstm_reg_a_glove_ann_loaded.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [ ]:
dev_loss, dev_acc = lstm_reg_a_glove_ann_loaded.evaluate(X_test_lstm_reg_a_glove_padded, y_test_lstm_reg_a_glove, verbose=1)

print(f"Training MSE: {np.sqrt(metrics.mean_squared_error(y_train_lstm_reg_a_glove, lstm_reg_a_glove_ann_loaded.predict(X_train_lstm_reg_a_glove_padded)))}")
print(f"Test MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_a_glove, lstm_reg_a_glove_ann_loaded.predict(X_test_lstm_reg_a_glove_padded)))}")
print(f"Test R^2: {metrics.r2_score(y_test_lstm_reg_a_glove, lstm_reg_a_glove_ann_loaded.predict(X_test_lstm_reg_a_glove_padded))}")
print(f"Baseline MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_a_glove, 0*y_test_lstm_reg_a_glove))}")

In [ ]:
# summarize history for loss
plt.plot(lstm_reg_a_glove_history.history['loss'])
plt.plot(lstm_reg_a_glove_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Model B: Word Vectors + 30min Price History as Inputs

Method 1: Word2Vec

In [ ]:
X_lstm_reg_b_w2v = master_df.loc[:, ['lstm_text', 'prev_30_min_prices']
y_lstm_reg_b_w2v = master_df.loc[:, '60mins_price_diff_perc']*100

In [ ]:
X_train_lstm_reg_b_w2v, X_test_lstm_reg_b_w2v, y_train_lstm_reg_b_w2v, y_test_lstm_reg_b_w2v = train_test_split(X_lstm_reg_b_w2v, y_lstm_reg_b_w2v, test_size=0.33, random_state=2020)

def parse_price_history(price_history):
    result = [float(i) for i in price_history.strip('[').strip(']').replace(' ', '').split(',')]
    return result

X_train_price_history = X_train_lstm_reg_b_w2v.iloc[:, 1].apply(parse_price_history)
X_train_price_history = np.stack(X_train_price_history)
X_train_price_history = X_train_price_history.reshape(X_train_price_history.shape[0], X_train_price_history.shape[1],1)

X_test_price_history = X_test_lstm_reg_b_w2v.iloc[:, 1].apply(parse_price_history)
X_test_price_history = np.stack(X_test_price_history)
X_test_price_history = X_test_price_history.reshape(X_test_price_history.shape[0], X_test_price_history.shape[1],1)
    
# Word2Vec embeddings are the same as the one used in Model A
X_train_lstm_reg_b_w2v = [X_train_lstm_reg_a_w2v_padded, np.array(X_train_price_history)]
X_test_lstm_reg_b_w2v = [X_test_lstm_reg_a_w2v_padded, X_test_price_history]

In [ ]:
def create_lstm_reg_b_w2v(pretrained_weights, longest_sentence_len, price_history_shape):
    vocab_size, embedding_size = pretrained_weights.shape
    
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[pretrained_weights], trainable=False)(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='tanh')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [ ]:
lstm_reg_b_w2v_ann = create_lstm_reg_b_w2v(lstm_reg_a_w2v_weights, lstm_reg_a_w2v_longest_sentence_len)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
lstm_reg_b_w2v_ann.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
lstm_reg_b_w2v_ann.summary()

In [ ]:
plot_model(
    lstm_reg_b_w2v_ann,
    to_file="lstm_reg_b_w2v_ann.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB"
)

In [ ]:
now = dt.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

lstm_reg_b_w2v_checkpoint_filepath = f'./lstm_reg_b_w2v/lstm_reg_b_w2v_{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

lstm_reg_b_w2v_history = lstm_reg_b_w2v_ann.fit(X_train_lstm_reg_a_w2v_padded, y_train_lstm_reg_b_w2v, validation_split=0.33, epochs=50, callbacks=[model_checkpoint_callback])

In [ ]:
lstm_reg_b_w2v_ann_loaded = create_lstm_reg_b_w2v(lstm_reg_a_w2v_weights, lstm_reg_a_w2v_longest_sentence_len, (30,1,))
lstm_reg_b_w2v_ann_loaded.load_weights(lstm_reg_b_w2v_checkpoint_filepath)
lstm_reg_b_w2v_ann_loaded.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [ ]:
dev_loss, dev_acc = lstm_reg_b_w2v_ann_loaded.evaluate(X_test_lstm_reg_a_w2v_padded, y_test_lstm_reg_b_w2v, verbose=1)

print(f"Training MSE: {np.sqrt(metrics.mean_squared_error(y_train_lstm_reg_b_w2v, lstm_reg_b_w2v_ann_loaded.predict(X_train_lstm_reg_a_w2v_padded)))}")
print(f"Test MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_b_w2v, lstm_reg_b_w2v_ann_loaded.predict(X_test_lstm_reg_a_w2v_padded)))}")
print(f"Test R^2: {metrics.r2_score(y_test_lstm_reg_b_w2v, lstm_reg_b_w2v_ann_loaded.predict(X_test_lstm_reg_a_w2v_padded))}")
print(f"Baseline MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_b_w2v, 0*y_test_lstm_reg_b_w2v))}")

In [ ]:
# summarize history for loss
plt.plot(lstm_reg_b_w2v_history.history['loss'])
plt.plot(lstm_reg_b_w2v_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Method 2: Glove

In [ ]:
# Glove embeddings are the same as the one used in Model A
X_train_lstm_reg_b_glove = [X_train_lstm_reg_a_glove_padded, np.array(X_train_price_history)]
X_test_lstm_reg_b_glove = [X_test_lstm_reg_a_glove_padded, X_test_price_history]

In [ ]:
def create_lstm_reg_b_glove(longest_sentence_len, price_history_shape):
    global embedding_layer_glove
    # word vectors model
    model1_input = layers.Input(shape=longest_sentence_len, dtype='int32', name='sentence_index_input')
    model1 = embedding_layer_glove(model1_input)  
    model1 = layers.LSTM(4, return_sequences=True, name='model1_LSTM1')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout1')(model1)
    model1 = layers.LSTM(4, return_sequences=False, name='model1_LSTM2')(model1)
    model1 = layers.Dropout(0.25,name='model1_dropout2')(model1)
    
    # price history model
    model2_input = layers.Input(shape=price_history_shape, dtype='float32', name='price_history_input')
    model2 = layers.LSTM(4, return_sequences=True, name='model2_LSTM1')(model2_input)
    model2 = layers.Dropout(0.25,name='model2_dropout1')(model2)
    model2 = layers.LSTM(4, return_sequences=False, name='model2_LSTM2')(model2)
    model2 = layers.Dropout(0.25,name='model2_dropout2')(model2)
    
    model_concat = layers.concatenate([model1, model2])
    model_concat = layers.Dense(4,name='Dense',activation='tanh')(model_concat)
    model_concat = layers.Dropout(0.1)(model_concat)
    model_concat = layers.Dense(1,activation='linear')(model_concat)
    
    model = keras.models.Model(inputs=[model1_input, model2_input], outputs = model_concat)
    
    return model

In [ ]:
lstm_reg_b_glove_ann = create_lstm_reg_b_glove(lstm_reg_a_glove_longest_sentence_len, (30,1,))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
lstm_reg_b_glove_ann.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
lstm_reg_b_glove_ann.summary()

In [ ]:
plot_model(
    lstm_reg_b_glove_ann,
    to_file="lstm_reg_b_glove_ann.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB"
)

In [ ]:
now = dt.now()
dt_string = now.strftime("%d%m%Y %H%Mh")

lstm_reg_b_glove_checkpoint_filepath = f'./lstm_reg_b_glove/lstm_reg_b_glove_{dt_string}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose = 1,
    save_best_only=True) 

lstm_reg_b_glove_history = lstm_reg_b_glove_ann.fit(X_train_lstm_reg_a_glove_padded, y_train_lstm_reg_b_glove, validation_split=0.33, epochs=50, callbacks=[model_checkpoint_callback])

In [ ]:
lstm_reg_b_glove_ann_loaded = create_lstm_reg_b_glove(lstm_reg_b_glove_longest_sentence_len, (30,1,))
lstm_reg_b_glove_ann_loaded.load_weights(lstm_reg_b_glove_checkpoint_filepath)
lstm_reg_b_glove_ann_loaded.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

In [ ]:
dev_loss, dev_acc = lstm_reg_b_glove_ann_loaded.evaluate(X_test_lstm_reg_a_glove_padded, y_test_lstm_reg_b_glove, verbose=1)

print(f"Training MSE: {np.sqrt(metrics.mean_squared_error(y_train_lstm_reg_b_glove, lstm_reg_b_glove_ann_loaded.predict(X_train_lstm_reg_a_glove_padded)))}")
print(f"Test MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_b_glove, lstm_reg_b_glove_ann_loaded.predict(X_test_lstm_reg_a_glove_padded)))}")
print(f"Test R^2: {metrics.r2_score(y_test_lstm_reg_b_glove, lstm_reg_b_glove_ann_loaded.predict(X_test_lstm_reg_a_glove_padded))}")
print(f"Baseline MSE: {np.sqrt(metrics.mean_squared_error(y_test_lstm_reg_b_glove, 0*y_test_lstm_reg_b_glove))}")

In [ ]:
# summarize history for loss
plt.plot(lstm_reg_b_glove_history.history['loss'])
plt.plot(lstm_reg_b_glove_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# 2. Classification

## Naive Bayes

Method 1: Text-Based

In [20]:
df1 = pd.read_csv("/Users/lavanyajindal/Documents/GitHub/MLProject/master_data.csv")

In [21]:
df1.shape

(2246, 23)

In [22]:
df1.head()

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,...,text_sentiment_analysis,text_LSTM,60mins_price_diff_abs,60mins_price_diff_perc,45mins_price_diff_abs,45mins_price_diff_perc,30mins_price_diff_abs,30mins_price_diff_perc,15mins_price_diff_abs,15mins_price_diff_perc
0,9.353400e+17,Thank you Rand! https://t.co/NvPeleVmub,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,...,Thank you Rand!,thank you rand,0.385000,0.001475,0.115000,0.000440,-0.010000,-0.000038,0.050000,0.000191
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,...,"Join me live from Fort Myer in Arlington, Virg...",join me live from fort myer in arlington virginia,0.590000,0.002421,0.590000,0.002421,-0.090000,-0.000369,-0.040000,-0.000164
2,8.939700e+17,Thank you Nicole! https://t.co/KlWN05uFOx,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,...,Thank you Nicole!,thank you nicole,0.065000,0.000271,0.015000,0.000063,0.085000,0.000354,0.065000,0.000271
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,...,Thank you to Shawn Steel for the nice words on .,thank you to shawn steel for the nice word on,0.142857,0.000603,0.096000,0.000405,0.130000,0.000549,0.035000,0.000148
4,8.787250e+17,MAKE AMERICA GREAT AGAIN!,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,...,MAKE AMERICA GREAT AGAIN!,make america great again,0.005714,0.000023,0.004286,0.000018,0.002857,0.000012,0.001429,0.000006


In [43]:
### get the range of the absolute diff in price change after tweet

In [23]:
min(df1.loc[:,"60mins_price_diff_abs"])

-3.989999999999952

In [24]:
max(df1.loc[:,"60mins_price_diff_abs"])

2.9699999999999704

In [ ]:
#add a target column for the y label {-1,0,1}

In [25]:
target_value = []

for i in range(0, len(df1)):
    
    if df1.loc[i,'60mins_price_diff_abs'] >= 0.020000:
        target_value.append(1)

    elif df1.loc[i,'60mins_price_diff_abs'] <= -0.003393:
        target_value.append(-1)

    else:
        target_value.append(0)
    

In [26]:
df1.loc[:, 'target'] = target_value

In [27]:
df1.head()

,id,text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,...,text_LSTM,60mins_price_diff_abs,60mins_price_diff_perc,45mins_price_diff_abs,45mins_price_diff_perc,30mins_price_diff_abs,30mins_price_diff_perc,15mins_price_diff_abs,15mins_price_diff_perc,target
0,9.353400e+17,Thank you Rand! https://t.co/NvPeleVmub,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,...,thank you rand,0.385000,0.001475,0.115000,0.000440,-0.010000,-0.000038,0.050000,0.000191,1
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,...,join me live from fort myer in arlington virginia,0.590000,0.002421,0.590000,0.002421,-0.090000,-0.000369,-0.040000,-0.000164,1
2,8.939700e+17,Thank you Nicole! https://t.co/KlWN05uFOx,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,...,thank you nicole,0.065000,0.000271,0.015000,0.000063,0.085000,0.000354,0.065000,0.000271,1
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,...,thank you to shawn steel for the nice word on,0.142857,0.000603,0.096000,0.000405,0.130000,0.000549,0.035000,0.000148,1
4,8.787250e+17,MAKE AMERICA GREAT AGAIN!,134210,36346,2017-06-24 21:23:00,2017-06-25 05:23:00,2017-06-25,05:23:00,5,2017,...,make america great again,0.005714,0.000023,0.004286,0.000018,0.002857,0.000012,0.001429,0.000006,0


In [ ]:
#drop missing values before fitting the models

In [28]:
df1.dropna(inplace=True)

In [29]:
df1.shape

(2197, 24)

In [ ]:
#define the x and y variables

In [30]:
X = df1.text_LSTM
y =df1.target

In [31]:
#Train-test split
#test size is 20% of the dataset
x_train,x_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
#get an understanding of the class distribution in training and test

In [32]:
y_train.value_counts()

-1    706
 1    698
 0    353
Name: target, dtype: int64

In [33]:
y_test.value_counts()

 1    175
-1    171
 0     94
Name: target, dtype: int64

In [72]:
#Naive-Bayes classification
#more no. of features doesnt seem to improve accuracy after 5000
#better results using cv + tfidftrransformer
pipe1_ = Pipeline([('vect', CountVectorizer(stop_words="english",max_features=1000)), ('tfidf', TfidfTransformer()), ('model1', MultinomialNB())])
#pipe3 = Pipeline([('vect', CountVectorizer(stop_words="english")),  ('model', MultinomialNB())])
model_nb1 = pipe1_.fit(x_train, y_train)
nb_pred1 = model_nb1.predict(x_test)

print("Accuracy of Naive Bayes Classifier: {}%".format(round(accuracy_score(y_test, nb_pred1)*100,2)))
print("\nConfusion Matrix of Naive Bayes Classifier:\n")
print(confusion_matrix(y_test, nb_pred1))
print("\nClassification Report of Naive Bayes Classifier:\n")
print(classification_report(y_test, nb_pred1))

Accuracy of Naive Bayes Classifier: 37.5%

Confusion Matrix of Naive Bayes Classifier:

[[75  5 91]
 [50  1 43]
 [82  4 89]]

Classification Report of Naive Bayes Classifier:

              precision    recall  f1-score   support

          -1       0.36      0.44      0.40       171
           0       0.10      0.01      0.02        94
           1       0.40      0.51      0.45       175

    accuracy                           0.38       440
   macro avg       0.29      0.32      0.29       440
weighted avg       0.32      0.38      0.34       440



In [74]:
param_grid = {
    'model1__alpha' : [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
}


# Instantiate the grid search model
grid_search = GridSearchCV(pipe1_, 
                           n_jobs = -1,cv = 3,param_grid=param_grid, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy * 100)
    print()
    print('Confusion Matrix:\n', confusion_matrix(y_test,y_pred))
    print()
    print('Classification Report:\n', classification_report(y_test, y_pred))

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'model1__alpha': 0.01}

Accuracy: 37.27272727272727

Confusion Matrix:
 [[67 17 87]
 [51  4 39]
 [69 13 93]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.36      0.39      0.37       171
           0       0.12      0.04      0.06        94
           1       0.42      0.53      0.47       175

    accuracy                           0.37       440
   macro avg       0.30      0.32      0.30       440
weighted avg       0.33      0.37      0.35       440



[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.6s finished


Method 2: Feature-Based

In [ ]:
# Vedant to do

## Logistic Regression

Method 1: Text-Based

In [75]:
#Logistic regression classification
pipe2_ = Pipeline([('vect', CountVectorizer(stop_words="english", min_df=3, ngram_range=(1,1), max_features=200)), ('tfidf', TfidfTransformer()), ('model2', LogisticRegression())])
#models must be in the correct order, where previous models must have fit and transform functionality while 
#last model must only have fit
model_lr1 = pipe2_.fit(x_train, y_train)
lr_pred1 = model_lr1.predict(x_test)

print("Accuracy of Logistic Regression Classifier: {}%".format(round(accuracy_score(y_test, lr_pred1)*100,2)))
print("\nConfusion Matrix of Logistic Regression Classifier:\n")
print(confusion_matrix(y_test, lr_pred1))
print("\nCLassification Report of Logistic Regression Classifier:\n")
print(classification_report(y_test, lr_pred1))

Accuracy of Logistic Regression Classifier: 41.14%

Confusion Matrix of Logistic Regression Classifier:

[[84  9 78]
 [40  3 51]
 [75  6 94]]

CLassification Report of Logistic Regression Classifier:

              precision    recall  f1-score   support

          -1       0.42      0.49      0.45       171
           0       0.17      0.03      0.05        94
           1       0.42      0.54      0.47       175

    accuracy                           0.41       440
   macro avg       0.34      0.35      0.33       440
weighted avg       0.37      0.41      0.38       440



In [84]:
param_grid = {
    'model2__penalty': ['l1', 'l2'],
    'model2__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'model2__solver': ['newton-cg', 'lbfgs'],
    'model2__max_iter': [500, 1000],
    'model2__multi_class': ['auto', 'multinomial']
}


# Instantiate the grid search model
grid_search = GridSearchCV(pipe2_, 
                           n_jobs = -1,cv=3,param_grid=param_grid, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy * 100)
    print()
    print('Confusion Matrix:\n', confusion_matrix(y_test,y_pred))
    print()
    print('Classification Report:\n', classification_report(y_test, y_pred))

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 112 candidates, totalling 336 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    8.6s


{'model2__C': 1, 'model2__max_iter': 500, 'model2__multi_class': 'auto', 'model2__penalty': 'l2', 'model2__solver': 'newton-cg'}

Accuracy: 41.13636363636364

Confusion Matrix:
 [[84  9 78]
 [40  3 51]
 [75  6 94]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.42      0.49      0.45       171
           0       0.17      0.03      0.05        94
           1       0.42      0.54      0.47       175

    accuracy                           0.41       440
   macro avg       0.34      0.35      0.33       440
weighted avg       0.37      0.41      0.38       440



[Parallel(n_jobs=-1)]: Done 336 out of 336 | elapsed:   10.4s finished


Method 2: Feature-Based

In [ ]:
# Vedant to do

## Random Forest

Method 1: Text-Based

In [78]:
#Random Forest classification
pipe3_ = Pipeline([('vect', CountVectorizer(stop_words="english", min_df=3, ngram_range=(1,1), max_features=200)), ('tfidf', TfidfTransformer()), ('model3', RandomForestClassifier())])
#models must be in the correct order, where previous models must have fit and transform functionality while 
#last model must only have fit
model_lr1 = pipe3_.fit(x_train, y_train)
lr_pred1 = model_lr1.predict(x_test)

print("Accuracy of Logistic Regression Classifier: {}%".format(round(accuracy_score(y_test, lr_pred1)*100,2)))
print("\nConfusion Matrix of Logistic Regression Classifier:\n")
print(confusion_matrix(y_test, lr_pred1))
print("\nCLassification Report of Logistic Regression Classifier:\n")
print(classification_report(y_test, lr_pred1))

Accuracy of Logistic Regression Classifier: 40.0%

Confusion Matrix of Logistic Regression Classifier:

[[84 14 73]
 [47  9 38]
 [80 12 83]]

CLassification Report of Logistic Regression Classifier:

              precision    recall  f1-score   support

          -1       0.40      0.49      0.44       171
           0       0.26      0.10      0.14        94
           1       0.43      0.47      0.45       175

    accuracy                           0.40       440
   macro avg       0.36      0.35      0.34       440
weighted avg       0.38      0.40      0.38       440



In [85]:
pipe3_.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'model3', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'model3__bootstrap', 'model3__ccp_alpha', 'model3__class_weight', 'model3__criterion', 'model3__max_depth', 'model3__max_features', 'model3__max_leaf_nodes', 'model3__max_samples', 'model3__min_impurity_decrease', 'model3__min_impurity_split', 'model3__min_samples_leaf', 'model3__min_samples_split', 'model3__min_weight_fraction_leaf', 'model3__n_estimators', 'model3__n_jobs', 'model3__oob_score', 'model3__random_state', 'model3__verbose', 'model3__warm_start'])

In [96]:
param_grid = {
    'model3__bootstrap': [True],
    'model3__max_leaf_nodes':[4,5,6],
    'model3__max_depth': [10,15],
    'model3__max_features': [2, 3],
    'model3__min_samples_leaf': [3, 4, 5, 6],
    #'model3__min_samples_split': [3, 4, 5, 6],
    'model3__n_estimators': [1000, 1300]
}

# Instantiate the grid search model
grid_search = GridSearchCV(pipe3_, 
                           n_jobs = -1,cv=3,param_grid=param_grid, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy * 100)
    print()
    print('Confusion Matrix:\n', confusion_matrix(y_test,y_pred))
    print()
    print('Classification Report:\n', classification_report(y_test, y_pred))

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 96 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:  5.3min finished


{'model3__bootstrap': True, 'model3__max_depth': 10, 'model3__max_features': 3, 'model3__max_leaf_nodes': 5, 'model3__min_samples_leaf': 6, 'model3__n_estimators': 1000}

Accuracy: 40.909090909090914

Confusion Matrix:
 [[108   0  63]
 [ 51   0  43]
 [103   0  72]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.41      0.63      0.50       171
           0       0.00      0.00      0.00        94
           1       0.40      0.41      0.41       175

    accuracy                           0.41       440
   macro avg       0.27      0.35      0.30       440
weighted avg       0.32      0.41      0.36       440



/Users/lavanyajindal/Desktop/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Method 2: Feature-Based

In [ ]:
# Vedant to do

## Support Vector Machine (SVM)

Method 1: Text-Based

In [93]:
#Support Vector classification
#accuracy score is better without applying the TFIDFTRANSFORMER()
pipe4_ = Pipeline([('vect', CountVectorizer(stop_words="english")), ('tfidf', TfidfTransformer()), ('model4', SVC())])
#pipe2 = Pipeline([('vect', CountVectorizer(stop_words="english")),  ('model', LinearSVC())])
model_svc1 = pipe4_.fit(x_train, y_train)
svc_pred1 = model_svc1.predict(x_test)

print("Accuracy of SVM Classifier: {}%".format(round(accuracy_score(y_test, svc_pred1)*100,2)))
print("\nConfusion Matrix of SVM Classifier:\n")
print(confusion_matrix(y_test, svc_pred1))
print("\nClassification Report of SVM Classifier:\n")
print(classification_report(y_test, svc_pred1))

Accuracy of SVM Classifier: 41.36%

Confusion Matrix of SVM Classifier:

[[88  3 80]
 [57  1 36]
 [81  1 93]]

Classification Report of SVM Classifier:

              precision    recall  f1-score   support

          -1       0.39      0.51      0.44       171
           0       0.20      0.01      0.02        94
           1       0.44      0.53      0.48       175

    accuracy                           0.41       440
   macro avg       0.34      0.35      0.32       440
weighted avg       0.37      0.41      0.37       440



In [94]:
pipe4_.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'model4', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'model4__C', 'model4__break_ties', 'model4__cache_size', 'model4__class_weight', 'model4__coef0', 'model4__decision_function_shape', 'model4__degree', 'model4__gamma', 'model4__kernel', 'model4__max_iter', 'model4__probability', 'model4__random_state', 'model4__shrinking', 'model4__tol', 'model4__verbose'])

In [95]:
param_grid = {
    'model4__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'model4__C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    'model4__gamma': ['scale', 'auto']
}
grid_search = GridSearchCV(pipe4_, 
                           n_jobs = -1,cv=3,param_grid=param_grid, verbose = 2, scoring = 'f1_weighted')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
print()

best_grid = grid_search.best_estimator_

def evaluate(model, test_features, test_labels):
    y_pred = model.predict(test_features)
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy * 100)
    print()
    print('Confusion Matrix:\n', confusion_matrix(y_test,y_pred))
    print()
    print('Classification Report:\n', classification_report(y_test, y_pred))

evaluate(best_grid,x_test,y_test)

Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:   45.7s finished


{'model4__C': 10, 'model4__gamma': 'scale', 'model4__kernel': 'linear'}

Accuracy: 37.72727272727273

Confusion Matrix:
 [[76 25 70]
 [43 16 35]
 [72 29 74]]

Classification Report:
               precision    recall  f1-score   support

          -1       0.40      0.44      0.42       171
           0       0.23      0.17      0.20        94
           1       0.41      0.42      0.42       175

    accuracy                           0.38       440
   macro avg       0.35      0.35      0.34       440
weighted avg       0.37      0.38      0.37       440



Method 2: Feature-Based

In [ ]:
# Vedant to do

## Artificial Neural Network (ANN)

In [5]:
# Brian to do